In [1]:
from open_clip import create_model_from_pretrained, get_tokenizer
import h5py
from tqdm import tqdm
import json
import torch

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.0+cu116
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
with open("data/MSVD/features/MSVD_annotations.json") as f:
    data = json.load(f)

res = {}
for key in data.keys():
    seen = set()
    res[key] = {}
    res[key]['entities']  = data[key]['class']
    res[key]['relations'] = []
    for h, r, t in data[key]['sorted_triplets']:
        if r not in seen:
            seen.add(r)
            res[key]['relations'].append(r)

In [ ]:
model, preprocess = create_model_from_pretrained('hf-hub:apple/DFN2B-CLIP-ViT-B-16') #xclip
tokenizer = get_tokenizer('ViT-B-16')

model.cuda()
print()

In [4]:
mapping = {}
with open("../Dataset/MSVD/captions/youtube-mapping.txt", "r") as f:
    for line in f:
        part = line.split(" ")
        mapping[part[1].strip("\n")] = part[0].strip()

In [6]:
with h5py.File("data/MSVD/features/MSVD_semantic_clip.hdf5", "w") as fs:
    for key in tqdm(res.keys()):
        video_id = mapping[key]
        with torch.no_grad():
            ent_tokens = tokenizer(res[key]['entities'])
            ent_embedding = model.encode_text(ent_tokens.cuda())
            ent_embedding = ent_embedding / ent_embedding.norm(dim=-1, keepdim=True)
            ent_embedding = ent_embedding.cpu()

            rel_tokens = tokenizer(res[key]['relations'])
            rel_embedding = model.encode_text(rel_tokens.cuda())
            rel_embedding = rel_embedding / rel_embedding.norm(dim=-1, keepdim=True)
            rel_embedding = rel_embedding.cpu()

            embeddings= torch.cat([ent_embedding, rel_embedding])
            fs.create_dataset(video_id, data=embeddings)

100%|██████████| 1970/1970 [04:26<00:00,  7.39it/s]
